In [229]:
#ivory - 'https://www.ivory.co.il/%D7%98%D7%90%D7%91%D7%9C%D7%98%D7%99%D7%9D.html'
#  zap - 'https://www.zap.co.il/models.aspx?sog=c-tabletpc'
#  ksp - 'https://ksp.co.il/web/cat/270..1045'
# bug - 'https://www.bug.co.il/tablets/'
# מחסני חשמל- 'https://www.payngo.co.il/computers-pcs/smartphones/tablets.html?p=4'
# wisebuy- 'https://www.wisebuy.co.il/products.aspx?category=c-tabletpc'
#go mobile - 'https://www.gomobile.co.il/25137-%D7%98%D7%90%D7%91%D7%9C%D7%98%D7%99%D7%9D'
from bs4 import BeautifulSoup
import sqlite3
import requests
import pandas as pd
from random import randint
import time
from time import sleep
from typing import List
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, f1_score

In [230]:
def sleepms(milliseconds: int):
    seconds = 0.001 * milliseconds
    sleep(seconds)
    
def random_wait():
    start = time.time()
    sleepms(randint(1, 250))
def getIvoryHeaders():
    url='https://www.ivory.co.il/catalog.php?id=55718'
    page = requests.get(url, headers = get_fake_user_agent())
    soup= BeautifulSoup(page.content,'html.parser')
    Header=['price']
    product=soup.findAll('li',attrs={'class':'col-md-12 col-12'})
    for prod in product:
         Header.append(prod.find('div',style='vertical-align: middle; width:40%; display:inline-block; padding-top: 5px; padding-bottom: 5px;color: #000000;').text.strip())
    return Header
def get_fake_user_agent():
    return {"User-Agent":"PostmanRuntime/7.29.0"}
       
def change_CSV_File_Headers(df):
    df.rename(columns={"מותג": "Brand"}, inplace=True)
    print('changed')
    df.rename(columns={"דגם": "Model"}, inplace=True)
    df.rename(columns={"תצוגה": "Display"}, inplace=True)
    df.rename(columns={"רזולוציית מסך": "Screen resolution"}, inplace=True)
    df.rename(columns={"מעבד": "Processor"}, inplace=True)
    df.rename(columns={"קישוריות": "connectivity"}, inplace=True)
    df.rename(columns={"כ.רשת אלחוטי": "Wireless network card"}, inplace=True)
    df.rename(columns={"נפח כולל (הסבר בתחתית העמוד)": "Total volume"}, inplace=True)
    df.rename(columns={"זכרון RAM": "RAM Memory"}, inplace=True)
    df.rename(columns={"חריץ הרחבת זכרון (כרטיס זכרון ניתן לרכישה בנפרד)": "Memory expansion slot"}, inplace=True)
    df.rename(columns={"מצלמה קידמית": "Front camera"}, inplace=True)
    df.rename(columns={"מצלמה אחורית": "Rear camera"}, inplace=True)
    df.rename(columns={"מערכת הפעלה": "Operating system"}, inplace=True)
    df.rename(columns={"חיבור טעינה והעברת נתונים": "Charging and data transfer connection"}, inplace=True)
    df.rename(columns={"חיבור אוזניות": "Connecting headphones"}, inplace=True)
    df.rename(columns={"צבע": "Color"}, inplace=True)
    df.rename(columns={"משקל כ-": "Weight"}, inplace=True)
    df.rename(columns={"שונות": "Miscellaneous"}, inplace=True)
    df.rename(columns={"אחריות": "Warranty"}, inplace=True)
    df.to_csv("ivoryData.csv",index=False)



In [232]:
def getIvoryData(url,headers):
    page=requests.get(url)
    soup=BeautifulSoup(page.content,'html.parser')
    product=soup.findAll('li',class_='col-md-12 col-12')
    data={}
    for prod in product:
         temp=prod.find('b').text
         valtemp=prod.find('div',style='vertical-align: middle; width:59%; display:inline-block; padding-top: 5px; padding-bottom: 5px; color: #000000;').text.strip()
         if(temp in headers):
            data.update({temp:valtemp})
         else:
            data[temp]=None
    return data

In [237]:
def all_ivory_files_to_csv():
    url='https://www.ivory.co.il/catalog.php?act=cat&id=2592'
    page = requests.get(url, headers = get_fake_user_agent())
    soup= BeautifulSoup(page.content,'html.parser')
    links=soup.find_all('a',class_='row product-anchor')
    prices=soup.findAll(class_='price')
    tabletPrice=[]
    TabltPriceToCol=[]
    for price in prices:
        tabletPrice.append(price.text.strip())  
    column=getIvoryHeaders()
    data=[]
    for link in links:
        data.append(getIvoryData(link.get('href'),column))
    df=pd.DataFrame(data,columns=column)
    df['price']=tabletPrice
    change_CSV_File_Headers(df)